In [1]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Mindsay/mathieu/Benter-Project

/home/mathieu/Mindsay/mathieu/Benter-Project


In [ ]:
%matplotlib inline
import collections
import functools

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from joblib import Memory

from utils import import_data
from utils import wagering_stategy
from utils import permutations
from winning_horse_models.logistic_regression import LogisticRegressionModel
from constants import PMU_BETTINGS, SOURCE_PMU
from utils.wagering_stategy import compute_expected_return, compute_scenario, plot_scenario
import wagering_stategies
from wagering_stategies import factories as wagering_factories
from constants import CACHE_DIR

from matplotlib import style
style.use('ggplot')

memory = Memory(location=CACHE_DIR, verbose=0)

winning_model = LogisticRegressionModel.load_model()

In [4]:
source = "PMU"
n_horses =16

In [ ]:
x, y, _ = import_data.get_races_per_horse_number(
    source=source,
    n_horses=n_horses,
    on_split="train",
    x_format="sequential_per_horse",
    y_format="rank",
)

In [6]:
min_horse, max_horse = import_data.get_min_max_horse(source=source)

In [21]:
rank=y

In [7]:
n_horses = 3

In [18]:
permutation=(1,2)

In [22]:
rank[:, permutation]

array([[2., 3.],
       [3., 2.],
       [2., 3.],
       [2., 1.],
       [2., 3.],
       [1., 3.],
       [2., 1.],
       [3., 2.],
       [2., 1.],
       [1., 3.],
       [2., 3.],
       [2., 1.],
       [2., 3.],
       [1., 3.],
       [2., 1.],
       [2., 3.],
       [2., 3.],
       [3., 1.],
       [2., 1.],
       [3., 1.],
       [2., 3.],
       [3., 1.],
       [2., 1.],
       [2., 3.],
       [1., 3.],
       [2., 3.],
       [2., 3.],
       [1., 3.],
       [3., 2.]], dtype=float32)

In [10]:
%%time
cut_permuted_x = np.concatenate([x[:,cut_permutation,:] for cut_permutation in permutations.get_n_cut_permutations(sequence=range(x.shape[1]), n=10, r=n_horses)])

CPU times: user 788 µs, sys: 34 µs, total: 822 µs
Wall time: 868 µs


In [11]:
cut_permuted_x.shape

(2710, 3, 49)

In [95]:
permuted_x.shape

(145, 3, 49)

In [99]:
np.reshape(a=permuted_x, newshape=(permuted_x.shape[0], -1),order="F").shape

(145, 147)

In [78]:
permutation = (1,2,0)

In [81]:
type(permutation.index(0))

int

In [91]:
np.concatenate(np.vectorize(lambda y_i: permutation.index(y_i))(y) for permutation

array([2, 2, 2, 1, 2, 0, 1, 2, 1, 0, 2, 1, 2, 0, 1, 2, 2, 1, 1, 1, 2, 1,
       1, 2, 0, 2, 2, 0, 2])

In [90]:
fv(y)

array([2, 2, 2, 1, 2, 0, 1, 2, 1, 0, 2, 1, 2, 0, 1, 2, 2, 1, 1, 1, 2, 1,
       1, 2, 0, 2, 2, 0, 2])

In [87]:
np.apply_along_axis(func1d=lambda y_i:print(y_i), axis=0, arr=y)

[0. 0. 0. 2. 0. 1. 2. 0. 2. 1. 0. 2. 0. 1. 2. 0. 0. 2. 2. 2. 0. 2. 2. 0.
 1. 0. 0. 1. 0.]


array(None, dtype=object)

In [53]:
permuted_y=np.concatenate(np.apply_along_axis(permutation.index)

In [54]:
get_perms(list(range(3)), N=10)

[(0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]

In [55]:
get_perms(list(range(3)), N=3)

[(2, 0, 1), (2, 1, 0), (0, 2, 1)]

In [57]:
get_n_permutations(list(range(3)), n=3)

[(2, 0, 1), (2, 1, 0), (0, 2, 1)]

In [19]:
n_horse_features = int(x.shape[1]/n_horses)

In [ ]:
permutation = [0,1,2]
permutation = [1,2,0]

In [10]:
y

array([0., 0., 0., 2., 0., 1., 2., 0., 2., 1., 0., 2., 0., 1., 2., 0., 0.,
       2., 2., 2., 0., 2., 2., 0., 1., 0., 0., 1., 0.], dtype=float32)

In [12]:
x[0,:]

array([ 0.        ,  0.        ,  0.        , -8.071549  , -8.071549  ,
       -8.071549  , -0.23596159, -0.23596159, -0.6498047 ,  0.        ,
        0.        ,  0.        ,  0.15278423,  0.15278423,  0.15278423,
        0.40554452,  0.40554452,  0.40554452,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -2.501967  ,
       -2.501967  , -2.501967  ,  0.        ,  0.        ,  0.        ,
       -1.1884238 , -0.70079905, -0.9446114 , -1.0369676 , -1.0369676 ,
       -1.0369676 , -0.76692   , -0.76692   , -0.5345543 ,  0.17303346,
        0.17303346,  0.17303346, -0.7898843 , -0.7898843 , -0.2987695 ,
        4.1373005 ,  4.1373005 ,  0.7519047 , -1.5515456 , -1.7249205 ,
       -0.5112963 ,  0.95560664,  0.07670319,  0.22820814, -0.9677058 ,
       -0.46857604, -0.4373541 ,  1.8462645 ,  0.11000843,  1.9087297 ,
       -1.6278287 , -0.9008076 , -1.6492289 ,  1.        ,  1.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.  